## Multi-Layer Perceptron (MLP) Ensemble Training Notebook

This notebook handles data generation, feature scaling, model architecture definition, and training for the multi-model ensemble. It is designed to be run sequentially.

In [ ]:
# --- 1. IMPORTS AND SETUP ---
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from datetime import datetime, timedelta
import os

# Ensure your stars_utils.py file is in the same directory
import stars_utils

# Set seed globally for reproducibility
RANDOM_SEED = 42
tf.keras.utils.set_random_seed(RANDOM_SEED)
os.makedirs('models', exist_ok=True)
print('Setup complete. TensorFlow version: ' + tf.version)

In [ ]:
# --- 2. CONFIGURATION ---
TARGET_PLANET = 'mars'
NUM_MODELS = 3 
EPOCHS = 5000   
TEST_SIZE = 0.2  
BATCH_SIZE = 32

# Time Range for Data Generation (Training Data)
START_DATE = datetime(1970, 1, 1)
END_DATE = datetime(2025, 1, 1)
TIME_STEP = timedelta(days=7) # Weekly data

# File Path for Scaler Persistence
MODEL_DIR = 'models'
SCALER_FILEPATH = os.path.join(MODEL_DIR, 'feature_scaler.pkl')

print('Configuration loaded.')

In [ ]:
# --- 3. DATA GENERATION, SPLIT, AND SCALING ---

print('1. Generating data for ' + TARGET_PLANET.capitalize() + '...')

# Use the utility function to get the raw ephemeris data
df_raw = stars_utils.generate_planetary_ephemeris_df(
    target_planet=TARGET_PLANET, 
    start_date=START_DATE, 
    end_date=END_DATE, 
    time_step=TIME_STEP
)

X = df_raw[['Julian_Date']] 
y = df_raw[['RA_deg', 'Dec_deg', 'Distance_AU']] 

# Chronological Train-Test Split
split_index = int(len(df_raw) * (1 - TEST_SIZE))
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Scaling the features (Julian Date)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
INPUT_SHAPE = X_train_scaled.shape[1] 

# Save the fitted scaler for later use in prediction/analysis notebooks
stars_utils.save_scaler(scaler, SCALER_FILEPATH)

# Create robust, optimized TensorFlow Datasets
AUTOTUNE = tf.data.AUTOTUNE
raw_train_dataset = tf.data.Dataset.from_tensor_slices((X_train_scaled, y_train.values))
raw_test_dataset = tf.data.Dataset.from_tensor_slices((X_test_scaled, y_test.values))
test_dataset = raw_test_dataset.batch(BATCH_SIZE).prefetch(AUTOTUNE)

print('Data split and scaled. Scaler saved.')

In [ ]:
# --- 4. MODEL CONFIGURATION AND CALLBACKS ---
models = []
y_pred_list = []

early_stopping_callback = EarlyStopping(
    monitor='val_loss', 
    patience=150,
    restore_best_weights=True 
)

lr_on_plateau_callback = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5,
    patience=50,
    min_lr=1e-7
)

callbacks = [early_stopping_callback, lr_on_plateau_callback]
best_ensemble_rmse = float('inf')

print('Callbacks defined.')

In [ ]:
# --- 5. BUILD AND TRAIN THE INDIVIDUAL MLP MODELS ---
print('
2. Building and training ' + str(NUM_MODELS) + ' diverse MLP models...')

for i in range(NUM_MODELS):
    # CRITICAL: Clear session to ensure fresh start for each model
    tf.keras.backend.clear_session()
    
    # Use a different random seed and shuffle seed for diversity
    current_seed = RANDOM_SEED + i
    tf.keras.utils.set_random_seed(current_seed)

    # Dataset with unique shuffle for this model
    train_dataset = raw_train_dataset.shuffle(
        buffer_size=1024, 
        seed=current_seed  # Unique shuffle for diversity
    ).batch(BATCH_SIZE).prefetch(AUTOTUNE)

    # --- Model Architecture ---
    model = Sequential([
        Dense(128, activation='relu', kernel_regularizer=l2(0.0001), 
              input_shape=(INPUT_SHAPE,)), 
        
        Dense(256, activation='relu', kernel_regularizer=l2(0.0001)),
        
        Dense(128, activation='relu', kernel_regularizer=l2(0.0001)), 
        
        Dense(64, activation='relu', kernel_regularizer=l2(0.0001)),

        Dense(3, activation='linear') # Output for RA, Dec, Distance
    ])
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    print('
--- Training MLP Model ' + str(i+1) + ' (Seed: ' + str(current_seed) + ') ---')
    model.fit(
        train_dataset, 
        epochs=EPOCHS, 
        validation_data=test_dataset,
        callbacks=callbacks,
        verbose=1
    )
    
    models.append(model)
    
    # Generate predictions for the current model
    y_pred = model.predict(X_test_scaled)
    y_pred_list.append(y_pred)
    
    # Calculate and print the individual model's RMSE
    overall_rmse = np.sqrt(mean_squared_error(y_test.values, y_pred))
    
    print('Model ' + str(i+1) + ' Test Set RMSE: ' + '{:.6f}'.format(overall_rmse) + ' AU')
    
    # Save the individual model (using the modern .keras format)
    model_save_name = os.path.join(MODEL_DIR, 'mars_position_predictor_mm' + str(i) + '.keras')
    model.save(model_save_name)
    print('Saved model to: ' + model_save_name)


In [ ]:
# --- 6. ENSEMBLE PREDICTION AND EVALUATION ---
print('
3. Final Ensemble Evaluation...')

# The ensemble prediction is the simple average of all individual model predictions.
y_ensemble_pred_au = np.mean(y_pred_list, axis=0)

# Evaluate the ensemble performance
overall_ensemble_rmse = np.sqrt(mean_squared_error(y_test.values, y_ensemble_pred_au))

print('Overall Ensemble RMSE: ' + '{:.6f}'.format(overall_ensemble_rmse) + ' AU')

print('
Training and evaluation complete. Models and scaler saved to the 'models' directory.')